In [2]:
from sqlalchemy import create_engine
import pandas as pd
from fuzzywuzzy import process, fuzz
import geopy
from geopy.geocoders import Nominatim
from os import environ
from googlemaps import Client
import math

In [3]:
connection_url =  environ["ICEURL"]

In [52]:
status = pd.read_sql(con=connection_url, sql="SELECT * FROM status_daily")
locations = pd.read_sql(con=connection_url, sql="SELECT * FROM locations")
# for col in ['date_first_record', 'date_recent_record', 'created_at', 'updated_at' ]:
#     dataframe2[col] = (new_dataframe[col]).apply(lambda d: pd.to_datetime(str(d)))
# dataframe2.to_sql(name='locations', con=connection_url, if_exists='append', index=False)

In [54]:
status.sort_values(by=['date'],inplace=True)

In [55]:
print(pd.io.sql.get_schema(status,'status'))


CREATE TABLE "status" (
"id" INTEGER,
  "alien_number" TEXT,
  "date" DATE,
  "location_id" REAL,
  "agency" TEXT,
  "accompanied_status" TEXT,
  "created_at" TIMESTAMP,
  "updated_at" TIMESTAMP,
  "age_on_date" REAL
)


In [106]:
locations.head(5)

,id,type,agency,program_type,name,address1,address2,address3,city,state,...,contact_last_name,date_first_record,date_recent_record,notes,phone,fax,url,email,created_at,updated_at
0,4,ICE ERO Field Office,ICE,None,Atlanta Field Office,180 Ted Turner Dr. SW Suite 522,None,None,Atlanta,GA,...,None,2016-07-26,2016-07-29,None,(404) 893-1210,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,Atlanta.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:46.973700+00:00
1,5,ICE ERO Field Office,ICE,None,SND Field Office,None,None,None,None,None,...,None,2015-12-22,2017-01-22,None,None,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,None,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.152500+00:00
2,6,ICE ERO Field Office,ICE,None,San Diego Field Office,880 Front Street #2232,None,None,San Diego,CA,...,None,2015-08-08,2016-09-26,None,(619) 436-0410,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,SanDiego.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.333600+00:00
3,7,ICE ERO Field Office,ICE,None,Newark Field Office,970 Broad St. 11th Floor,None,None,Newark,NJ,...,None,2016-02-17,2016-05-06,None,(973) 645-3666,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,Newark.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.514800+00:00
4,8,ICE ERO Field Office,ICE,None,San Antonio Field Office,1777 NE Loop 410 Floor 15,None,None,San Antonio,TX,...,None,2015-07-26,2016-04-26,None,(210) 283-4750,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,SanAntonio.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.689000+00:00


In [88]:
sf =  status[status['location_id'].notnull()]

In [93]:
sf = sf[sf['date'] == sf['date'].max()]

In [97]:
sf['location_id'] = pd.to_numeric(sf['location_id'], downcast='unsigned')

In [117]:
status_locations = sf[['id','location_id']].merge(locations[['id','lat','lon']], left_on='location_id', right_on='id')

In [119]:
del status_locations['id_y']
status_locations.head()

,id_x,location_id,lat,lon
0,2294294,309,33.48138,-112.051393
1,1983689,309,33.48138,-112.051393
2,2376551,309,33.48138,-112.051393
3,2335613,309,33.48138,-112.051393
4,2339616,309,33.48138,-112.051393


In [122]:
from ipyleaflet import Map, Marker, MarkerCluster, basemap_to_tiles, basemaps, WMSLayer, LayersControl
import pandas as pd
import math

m = Map(
    center=(39.1, -94.578),
    zoom=4,
    attribution_control=False
)

markers=[]
for index, row in status_locations.iterrows():
    lat, lon = row [['lat','lon']]
    if not math.isnan(lat):
        markers.append(Marker(location=(lat,lon)))

marker_cluster = MarkerCluster(markers=markers)
m.add_layer(marker_cluster)
m.add_layer(basemaps.Esri.DeLorme)
nasa_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-09-13")
m.add_layer(nasa_layer)
m.add_control(LayersControl())
m

Map(attribution_control=False, basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom…

In [123]:
sf.head()

,id,alien_number,date,location_id,agency,accompanied_status,created_at,updated_at,age_on_date
1914290,2294294,9656FCCFBECF4C6684863CCE11EAD43B01BBA3710CD7F4...,2018-06-14,309,ORR,Unaccompanied,2018-07-23 06:50:54.590800+00:00,2018-07-23 06:50:54.590800+00:00,6543.0
1957385,2337389,AAE10CDA81352DCFBD4E61E802C82907D688AB5D6597EE...,2018-06-14,154,ORR,Unaccompanied,2018-07-23 06:51:56.713000+00:00,2018-07-23 06:51:56.713000+00:00,5937.0
1997666,2377670,BD9D5FA86814B9C63BCCDE53D4A4A0E01A07A95F7FD3E1...,2018-06-14,330,ORR,Unaccompanied,2018-07-23 06:53:08.944700+00:00,2018-07-23 06:53:08.944700+00:00,5829.0
1601994,1981998,0689704A8A85D634E89082167CAAAEE10B80BFB05C80DD...,2018-06-14,191,ORR,Unaccompanied,2018-07-23 06:42:48.542200+00:00,2018-07-23 06:42:48.542200+00:00,6256.0
1996670,2376674,BD2BCABCE98B814E0B1B2DF846E0C39BC11869B359E828...,2018-06-14,255,ORR,Unaccompanied,2018-07-23 06:52:54.402000+00:00,2018-07-23 06:52:54.402000+00:00,6094.0
